In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
import sys
import os

path = "/content/drive/MyDrive/NLP_Project_New"
sys.path.append(os.path.abspath(path))

In [29]:
import pandas as pd
from datetime import datetime
import os.path

import logging

logfile = f"{path}/logs/evaluate_model_adv.log"
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

file_handler = logging.FileHandler(logfile)
file_handler.setFormatter(formatter)
file_handler.setLevel(logging.INFO)

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
stream_handler.setLevel(logging.INFO)

logger = logging.getLogger("evaluate_model_adv.log")
logger.setLevel(logging.INFO)

logger.addHandler(file_handler)
logger.addHandler(stream_handler)

In [30]:
def helper(x: str) -> str:
    if x == "B-PER":
        return 1
    elif x == "I-PER":
        return 2
    elif x == "B-LOC":
        return 3
    elif x == "I-LOC":
        return 4
    else:
        return 0

In [31]:
  test_pred_path = f'{path}/advers_model_comparisons/perturb_7_comparison.tsv'

  df = pd.read_csv(test_pred_path, sep="\t")
  df.head()

,TOKEN,NER,de_core_news_sm--03-26-06-08-38,/content/drive/MyDrive/NLP_Project_New/models/sermons_60it--03-26-06-15-17,dbmdz/bert-base-german-cased--03-26-06-43-39,/content/drive/MyDrive/NLP_Project_New/models/bert_base_german_cased_finetuned_3ep--03-26-07-51-24,dbmdz/flair-historic-ner-onb--03-26-08-44-50,dbmdz/flair-historic-ner-onb_finetuned_3ep_2e-5--03-26-09-37-54
0,Darum,O,O,O,B-LOC,O,O,O
1,auch,O,O,O,B-PER,O,O,O
2,der,O,O,O,O,O,O,O
3,selige,O,O,O,B-LOC,O,O,O
4,Jivika,B-PER,O,O,O,B-PER,B-PER,B-PER


In [32]:
all_adv_test_results = []

for idx in range(1,8):
  now = datetime.now()
  timestamp = now.strftime("%m-%d-%H-%M-%S")

  test_pred_path = f'{path}/advers_model_comparisons/perturb_{idx}_comparison.tsv'

  df = pd.read_csv(test_pred_path, sep="\t")

  logger.info(f"Initialised model evaluation. Predictions table loaded from location {test_pred_path}")

  attempt_results = []

  for attempt in df.columns[2:]:
    df_numeric = df[["NER", attempt]]
    df_numeric.loc[:,"NER"] = df_numeric["NER"].apply(helper)
    df_numeric.loc[:,attempt] = df_numeric[attempt].apply(helper)

    false_positives = 0
    false_negatives = 0
    true_positives = 0
    true_negatives = 0

    for index, row in df_numeric.iterrows():
        a = row['NER']
        b = row[attempt]

        if a < b:
            false_positives += 1
        elif a > b:
            false_negatives += 1
        elif (a == b) and (a > 0):
            true_positives += 1
        else:
            true_negatives += 1

    correct_predictions = true_positives + true_negatives
    false_predictions = false_positives + false_negatives
    all_predictions = correct_predictions + false_predictions

    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    accuracy = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives)
    f1 = (2 * precision * recall) / (precision + recall)

    logger.info(f"EVALUATION RESULTS:\nPrecision:\t\t{precision}\nRecall:\t\t\t{recall}\nAccuracy:\t\t{accuracy}\nF1-score:\t\t{f1}\n")

    eval_results = {'attempt': attempt,
                'time': timestamp,
                'model_name': attempt[:-16],
                'precision': precision,
                'recall': recall,
                'accuracy': accuracy,
                'F1-score': f1}

    attempt_results.append(eval_results)

    collated = {
        'Perturbation': idx,
        'Results': attempt_results
    }

  all_adv_test_results.append(collated)



2024-03-26 09:51:30,118 - evaluate_model_adv.log - INFO - Initialised model evaluation. Predictions table loaded from location /content/drive/MyDrive/NLP_Project_New/advers_model_comparisons/perturb_1_comparison.tsv
2024-03-26 09:51:30,118 - evaluate_model_adv.log - INFO - Initialised model evaluation. Predictions table loaded from location /content/drive/MyDrive/NLP_Project_New/advers_model_comparisons/perturb_1_comparison.tsv
INFO:evaluate_model_adv.log:Initialised model evaluation. Predictions table loaded from location /content/drive/MyDrive/NLP_Project_New/advers_model_comparisons/perturb_1_comparison.tsv
<ipython-input-32-2416f9b03b2c>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_numeric.loc[:,"NER"] = df_numeric["NER"].apply(helper

In [33]:
all_results = []

for test in all_adv_test_results:
  all_results.append(pd.DataFrame.from_dict(test["Results"]))

In [34]:
all_results[0]

,attempt,time,model_name,precision,recall,accuracy,F1-score
0,de_core_news_sm--03-26-06-08-38,03-26-09-51-29,de_core_news_sm,0.111707,0.512295,0.938512,0.183419
1,/content/drive/MyDrive/NLP_Project_New/models/...,03-26-09-51-29,/content/drive/MyDrive/NLP_Project_New/models/...,0.456140,0.103792,0.981180,0.169106
2,dbmdz/bert-base-german-cased--03-26-06-43-39,03-26-09-51-29,dbmdz/bert-base-german-cased,0.004064,0.109741,0.585069,0.007838
3,/content/drive/MyDrive/NLP_Project_New/models/...,03-26-09-51-29,/content/drive/MyDrive/NLP_Project_New/models/...,0.759680,0.575911,0.988969,0.655153
4,dbmdz/flair-historic-ner-onb--03-26-08-44-50,03-26-09-51-29,dbmdz/flair-historic-ner-onb,0.345361,0.538693,0.972838,0.420887
5,dbmdz/flair-historic-ner-onb_finetuned_3ep_2e-...,03-26-09-51-29,dbmdz/flair-historic-ner-onb_finetuned_3ep_2e-5,0.681704,0.272818,0.984310,0.389685


In [35]:
for idx in range(1,8):
  save_path = f'{path}/model_comparisons/model_results_adversarial/perturbation_{idx}.csv'

  all_results[idx - 1].to_csv(save_path, index=False)
  logger.info(f"Evaluation for perturbation {idx} saved at location {save_path}")

2024-03-26 09:53:43,834 - evaluate_model_adv.log - INFO - Evaluation for perturbation 1 saved at location /content/drive/MyDrive/NLP_Project_New/model_comparisons/model_results_adversarial/perturbation_1.csv
2024-03-26 09:53:43,834 - evaluate_model_adv.log - INFO - Evaluation for perturbation 1 saved at location /content/drive/MyDrive/NLP_Project_New/model_comparisons/model_results_adversarial/perturbation_1.csv
INFO:evaluate_model_adv.log:Evaluation for perturbation 1 saved at location /content/drive/MyDrive/NLP_Project_New/model_comparisons/model_results_adversarial/perturbation_1.csv
2024-03-26 09:53:43,852 - evaluate_model_adv.log - INFO - Evaluation for perturbation 2 saved at location /content/drive/MyDrive/NLP_Project_New/model_comparisons/model_results_adversarial/perturbation_2.csv
2024-03-26 09:53:43,852 - evaluate_model_adv.log - INFO - Evaluation for perturbation 2 saved at location /content/drive/MyDrive/NLP_Project_New/model_comparisons/model_results_adversarial/perturbat